# Tool Call

Procedure:
1. Define the tool
2. Then write its function description
3. Call the LLM using tools and inputs
4. Check if the LLM is asking for tool call in the response.output[0]
5. If it is calling tool then execute the function with args provided by the LLM. (args = json.loads(response.output[0].arguments)) and get the response from the defined tool 
6. Append the input message with LLM output message.append(response.output[0])
7. Append result message
    messages.append({
    "type":"function_call_output",
    "call_id":response.output[0].call_id,
    "output": str(output)
    })
8. Call LLM again with the message
    

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import requests
import json
load_dotenv()

True

In [2]:
client = OpenAI()


In [3]:
def get_weather(latitude, longitude):
    weather = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m")
    data = weather.json()
    return data['current']['temperature_2m']

In [4]:
messages = [
    {
        "role":"developer",
        "content": "What is the weather like in New Delhi? "
    }
]

tools = [
    {
        "type" : "function",
        "name" : "get_weather",
        "description":"fetches the weather information from the website",
        "parameters": {
            "type":"object",
            "properties":
            {
                "latitude":{"type":"number", "description":"Latitude of the location"},
                "longitude":{"type": "number", "description":"Longitude of the location"}
            },
            "required":["latitude", "longitude"],
            "additionalProperties":False
        },
        "strict":True

    }
]

In [5]:
response = client.responses.create(
    model = "gpt-4o",
    input = messages,
    tools = tools    
)

In [ ]:
print(response.output[0])

ResponseFunctionToolCall(arguments='{"latitude":28.6139,"longitude":77.209}', call_id='call_oSFRmRgayVwVFkeaL6xbd4Cr', name='get_weather', type='function_call', id='fc_6842e3e10964819dbd777f5100adb4000a3f53d0e20f2878', status='completed')


In [16]:
args = json.loads(response.output[0].arguments)

In [18]:
output = get_weather(args['latitude'], args['longitude'])
print(output)

34.5


In [19]:
messages.append(response.output[0])
messages.append({
    "type":"function_call_output",
    "call_id":response.output[0].call_id,
    "output": str(output)
})

In [20]:
print(messages)

[{'role': 'developer', 'content': 'What is the weather like in New Delhi? '}, ResponseFunctionToolCall(arguments='{"latitude":28.6139,"longitude":77.209}', call_id='call_oSFRmRgayVwVFkeaL6xbd4Cr', name='get_weather', type='function_call', id='fc_6842e3e10964819dbd777f5100adb4000a3f53d0e20f2878', status='completed'), {'type': 'function_call_output', 'call_id': 'call_oSFRmRgayVwVFkeaL6xbd4Cr', 'output': '34.5'}]


In [15]:
print(weather)

{'latitude': 28.6139, 'longitude': 77.209}


In [21]:
response2 = client.responses.create(
    model = "gpt-4o",
    tools = tools,
    input = messages
)

print(response2.output_text)

The current temperature in New Delhi is 34.5°C.


In [ ]:
[
    ResponseFunctionToolCall(
        arguments='{"latitude":28.6139,"longitude":77.209}',
        call_id='call_oSFRmRgayVwVFkeaL6xbd4Cr',
        name='get_weather',
        type='function_call',
        id='fc_6842e3e10964819dbd777f5100adb4000a3f53d0e20f2878',
        status='completed'
    )
]


In [ ]:
[
    {
        'role': 'developer',
        'content': 'What is the weather like in New Delhi?'
    },
  ResponseFunctionToolCall(
        arguments='{"latitude":28.6139,"longitude":77.209}',
        call_id='call_oSFRmRgayVwVFkeaL6xbd4Cr',
        name='get_weather',
        type='function_call',
        id='fc_6842e3e10964819dbd777f5100adb4000a3f53d0e20f2878',
        status='completed'
      ),
      {
          'type': 'function_call_output',
          'call_id': 'call_oSFRmRgayVwVFkeaL6xbd4Cr',
          'output': '34.5'
      }
]

# Agent 1: Simple Agent Loop

This agent sends a prompt (asking about the weather), then enters an agentic loop

At each turn, it calls the Responses API:
1. **If the response contains a tool call:** The agent executes the function (using our get_weather tool) and appends the function result to the conversation as a new message.

2. **If the response provides output text:** The agent stops, printing the final output.

Below is the code for the simple agent

In [14]:
while True:
    response = client.responses.create(
        model = "gpt-4.1-mini",
        input = messages,
        tools = tools
    )

    if response.output:
        for output_item in response.output:
            if output_item.type == "function_call":
                messages.append(output_item)

                tool_call = output_item
                args = json.loads(tool_call.arguments)
                result = get_weather(args['latitude'],args['longitude'])
                print(f"Executed {tool_call.name}: Result = {result} C")

                messages.append({
                    "type":"function_call_output",
                    "call_id": tool_call.call_id,
                    "output":str(result)
                })
    if response.output_text:
        print("Final Agent Output: ", response.output_text)
        break

    if not response.output:
        break


Executed get_weather: Result = 34.8 C
Final Agent Output:  The current temperature in New Delhi is approximately 34.8°C. If you want more details about the weather such as humidity, wind speed, or forecast, please let me know!


# Agent 2: Agent with Custom Objective Function

This agent uses a custom objective function to decide whether to continue looping. In this example, the objective function checks if the agent's output text contains the phrase "task complete".

The agent will continue to request responses (and execute any tool calls) until the objective is met. Note that this is a simplified demonstration intended for teaching purposes.


In [22]:
def get_weather(latitude, longitude):
    weather = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m")
    data = weather.json()
    return data['current']['temperature_2m']

In [23]:
tools = [
    {
        "type" : "function",
        "name" : "get_weather",
        "description":"fetches the weather information from the website",
        "parameters": {
            "type":"object",
            "properties":
            {
                "latitude":{"type":"number", "description":"Latitude of the location"},
                "longitude":{"type": "number", "description":"Longitude of the location"}
            },
            "required":["latitude", "longitude"],
            "additionalProperties":False
        },
        "strict":True

    }
]

In [24]:
def objective_met(search_count, max_search = 5):
    return search_count>max_search

In [28]:
# Initial conversation: developer sets the task, user gives the first city
messages= [
    {
        "role": "developer",
        "content": (
            "Your goal is to gather weather for at least 5 different cities. "
            "Once you've done that, respond with 'task complete'."
        )
    },
    {"role": "user", "content": "Search the weather in Berlin."}
]

search_count = 0

while True:
    response = client.responses.create(
        model="gpt-4o",
        input=messages,
        tools=tools
    )
    
    # 1) Handle any function calls (e.g. get_weather)
    for item in response.output or []:
        if getattr(item, 'type', None) == "function_call":
            messages.append(item)  # pass the function call back into context
            
            # parse and execute
            args = json.loads(item.arguments)
            temp = get_weather(args['latitude'], args['longitude'])
            print(f"Executed {item.name}: {temp}°C")
            
            # append function result
            messages.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": str(temp)
            })
            
            search_count += 1
    
    # 2) Check for assistant output_text
    if hasattr(response, 'output_text') and response.output_text:
        text = response.output_text
        print("Agent says:", text)
        messages.append({"role": "assistant", "content": text})
    
    # 3) Check objective
    if objective_met(search_count):
        print(f"Objective met: searched {search_count} cities. Task complete.")
        break
    
    # 4) If not done, prompt for the next city
    messages.append({
        "role": "user",
        "content": f"We've searched {search_count} so far. Please search another city."
    })

Executed get_weather: 21.5°C
Executed get_weather: 25.9°C
Executed get_weather: 20.5°C
Executed get_weather: 13.1°C
Executed get_weather: 22.0°C
Executed get_weather: 26.2°C
Objective met: searched 6 cities. Task complete.
